# Proyecto de ECI2019-NLP

Para este proyecto tenemos que replicar los resultados de este [paper](https://www.aclweb.org/anthology/N18-2017) (Gururangan et al., 2018). En el paper emplearon [FastText](https://fasttext.cc/) en el dataset de SNLI.

Lo primero para emplear FastText es instalar el paquete. [Aquí](https://fasttext.cc/docs/en/support.html) se explica el método de instalación del módulo de python:

```
$ git clone https://github.com/facebookresearch/fastText.git
$ cd fastText
$ sudo pip install .
$ # or :
$ sudo python setup.py install
```

Luego de eso se puede importar FastText simplemente haciendo:

```
$ python
Python 2.7.15 |(default, May  1 2018, 18:37:05)
Type "help", "copyright", "credits" or "license" for more information.
>>> import fasttext
>>>
```

---

### 1) Importamos los módulos que necesitamos para el baseline

In [1]:
import fasttext

from os import path
from read_data import it_labels, it_sentences

### 2) Preparamos los datasets

In [2]:
# Cargamos los archivos y instanciamos generadores para cada uno
train_data = it_sentences(open('snli_1.0_train_filtered.jsonl'))
train_labels = it_labels(open('snli_1.0_train_gold_labels.csv'))

FastText funciona tomando archivos de texto con cada línea siguiendo la siguiente estructura:

```
__label__<Label0> __label__<Label1> ... __label_<LabelN> <Sentence>
```

donde _Label0_,..., _LabelN_ son las etiquetas anotadas para la oración _Sentence_. En nuestro caso, _Label0_ será una de las siguientes: _neutral_, _contradiction_ o _entailment_. 

Con cada par (oración, etiqueta) en el conjunto de entrenamiento, vamos a crear un nuevo archivo con esta estructura para usar FastText. Si bien el paper no aclara si transforma todas las palabras a minúsculas, es una práctica común en NLP por lo que, cuando creamos el archivo, lo hacemos.

In [3]:
# Si el archivo ya existe, no lo creamos de nuevo.
if not path.exists('fasttext_train_file.txt'):
    ft_train_fl = open('fasttext_train_file.txt', 'w')
    for sent, label in zip(train_data, train_labels):
        # __label__<Label0> <Sentence>
        ft_train_fl.write('__label__{} {}\n'.format(label, sent))     
    ft_train_fl.close()

Comprobamos que el archivo esté creado correctamente.

In [3]:
fl = open('fasttext_train_file.txt')
for i in range(3):
    print(fl.readline())
fl.close()

__label__neutral A person is training his horse for a competition .

__label__contradiction A person is at a diner , ordering an omelette .

__label__entailment A person is outdoors , on a horse .



### 3) Entrenamos el clasificador de FastText

Según el Readme.md del repo de FastText: 

In order to train a text classifier using the method [described here](https://fasttext.cc/docs/en/references.html#bag-of-tricks-for-efficient-text-classification) (Joulin  et  al.,2017), we can use `fasttext.train_supervised` function like this:

```py
import fasttext

model = fasttext.train_supervised('data.train.txt')
```

El paper Joulin et al.,2017 está entre las citas de Gururangan et al., 2018 en la sección que habla del método que emplearon para obtener los resultados expuestos en la tabla 2.

También, segun el paper, emplearon un "off-the-shelf text classifier", por lo que dejamos todos los hiperparámetros del modelo con su valor por default.

In [4]:
# This is not the base plain supervised model described in (Gururangan et al., 2018)
model = fasttext.train_supervised('fasttext_train_file.txt', epoch=20, wordNgrams=2, loss='hs', )

In [5]:
print("Summary:")
print("Epochs:        {}".format(model.epoch))
print("Loss:          {}".format(model.loss))
print("Learning Rate: {}".format(model.lr))
print("Vector dim:    {}".format(model.dim))

Summary:
Epochs:        20
Loss:          loss_name.hs
Learning Rate: 0.1
Vector dim:    100


### 4) Evaluación de performance

Antes de evaluar performance, vamos a hacer comparar los labels más fácil empleando diccionarios.

In [6]:
pred2idx = dict(zip(model.labels, [0,1,2]))

print(pred2idx)

label2idx = {'entailment':0, 'contradiction':1, 'neutral':2}

print(label2idx)

{'__label__entailment': 0, '__label__contradiction': 1, '__label__neutral': 2}
{'entailment': 0, 'contradiction': 1, 'neutral': 2}


Cargamos el conjunto de validación

In [7]:
dev_data = it_sentences(open('snli_1.0_dev_filtered.jsonl'))
dev_labels = it_labels(open('snli_1.0_dev_gold_labels.csv'))

Evaluamos nuestro modelo con la métrica de precision

In [8]:
correct = 0
total = 0

for sent, label in zip(dev_data, dev_labels):
    pred,_ = model.predict(sent)
    total += 1
    # Comparamos el label obtenido con el label anotado
    correct += label2idx[label] == pred2idx[pred[0]]
    
print('Precision: {}'.format(float(correct)/total))

Precision: 0.5982523877260719


Vemos que el valor está lejos del valor reportado en el paper, puede que hayan más detalles finos que no están explicados en el paper en el que nos basamos.

Otro detalle importante: el entrenamiento de FastText tiene un componente aleatorio, por lo que cada nuevo entrenamiento que se haga da una performance distinta. Es posible que la diferencia de performance reportada y la obtenida aquí venga de la elección de la seed (que no he encontrado dónde se settea).

In [9]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA

import numpy as np
import random

## Probamos con Support Vector Machine y FastText Embeddings

In [10]:
def add2vocab(word2idx, data):
    for sent in data:
        for word in sent.lower().split():
            if word not in word2idx:
                word2idx[word] = len(word2idx)
        
class Fasttext_lookup():
    """
    Clase para manejar los embeddings de FastText
    """
    def __init__(self, embeddings_file, vocab, upper=1.0, lower=-1.0):
        self.upper=upper
        self.lower=lower
        self.fasttext = {}
        with open(embeddings_file, 'r') as embfl:
            for line in embfl:
                vals = line.strip().split()
                try:
                    word = vals[0]
                    if not(word in vocab):
                        continue
                    emb = np.array(vals[1:], dtype=np.float)
                    self.fasttext[word]=emb
                except:
                    pass
    
    def fasttext_vec(self, w):
        """
        Retorna el embeddings correspondiente a la palabra w, sino, un vector al azar entre upper y lower.
        """
        return self.fasttext.get(w, np.array([random.uniform(self.lower, self.upper) for i in range(300)]))

In [11]:
train_data = it_sentences(open('snli_1.0_train_filtered.jsonl'))
dev_data = it_sentences(open('snli_1.0_dev_filtered.jsonl'))
test_data = it_sentences(open('snli_1.0_test_filtered.jsonl'))

word2idx = {}
add2vocab(word2idx, train_data)
add2vocab(word2idx, dev_data)
add2vocab(word2idx, test_data)

In [12]:
print(len(word2idx))

47919


In [ ]:
embs_lookup = Fasttext_lookup('wiki.en.vec', word2idx)

In [45]:
word2idx.keys()

dict_keys(['a', 'person', 'is', 'training', 'his', 'horse', 'for', 'competition.', 'at', 'diner,', 'ordering', 'an', 'omelette.', 'outdoors,', 'on', 'horse.', 'they', 'are', 'smiling', 'their', 'parents', 'there', 'children', 'present', 'the', 'kids', 'frowning', 'boy', 'skates', 'down', 'sidewalk.', 'does', 'skateboarding', 'trick.', 'wearing', 'safety', 'equipment.', 'older', 'man', 'drinks', 'juice', 'as', 'he', 'waits', 'daughter', 'to', 'get', 'off', 'work.', 'flips', 'burger.', 'elderly', 'sits', 'in', 'small', 'shop.', 'some', 'women', 'hugging', 'vacation.', 'sleeping.', 'showing', 'affection.', 'people', 'eating', 'omelettes.', 'sitting', 'desks', 'school.', 'diners', 'restaurant.', 'drinking', 'juice.', 'two', 'restaurant', 'wine.', 'waiting', 'meal', 'arrive.', 'blond', 'getting', 'drink', 'of', 'water', 'from', 'fountain', 'park.', 'brown', 'shirt', 'reading', 'book', 'bench', 'park', 'fountain.', 'friends', 'scowl', 'each', 'other', 'over', 'full', 'dinner', 'table.', 'wom

### Generamos archivo de predicciones

In [10]:
test_data = it_sentences(open('snli_1.0_test_filtered.jsonl'))

In [11]:
test_fl = open('test_cls.txt', 'w')

for sent in test_data:
    pred,_ = model.predict(sent.lower())
    test_fl.write('{}\n'.format(pred[0]))
    
test_fl.close()